In [1]:
# install.packages("beepr")

In [2]:
library(ggplot2)
library(reshape2)

### Funciones Utilitarias (plot, lectura de archivos)

In [3]:
plot_fitness_over_time <- function(best_fitness, population_fitness, generations) {
  best_fitness_data <- data.frame(Generation = 1:generations, BestFitness = best_fitness)
  population_fitness_data <- melt(population_fitness)
  colnames(population_fitness_data) <- c("Individual", "Generation", "Fitness")
  
  p <- ggplot() +
    geom_violin(data = population_fitness_data, aes(x = Generation, y = Fitness, group = Generation), fill = "gray", alpha = 0.5) +
    geom_line(data = best_fitness_data, aes(x = Generation, y = BestFitness), color = "blue", size = 1) +
    labs(title = "Evolución del Fitness a lo Largo de las Generaciones", x = "Generación", y = "Fitness") +
    theme_minimal()
  
  print(p)
}


In [4]:
plot_fitness <- function(generations, best_fitness, average_fitness) {
  fitness_data <- data.frame(
    Generation = 1:generations,
    BestFitness = best_fitness,
    AverageFitness = average_fitness
  )
  
  ggplot(fitness_data, aes(x = Generation)) +
    geom_line(aes(y = BestFitness, color = "Best Fitness"), size = 1) +
    geom_line(aes(y = AverageFitness, color = "Average Fitness"), size = 1, linetype = "dashed") +
    scale_color_manual(values = c("Best Fitness" = "blue", "Average Fitness" = "green")) +
    labs(title = "Best and Average Fitness Over Generations", x = "Generations", y = "Fitness") +
    theme_minimal() +
    theme(
      plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
      axis.title = element_text(size = 12),
      legend.title = element_blank()
    )
}

# Función para graficar la diversidad de la población
plot_diversity <- function(generations, diversity) {
  diversity_data <- data.frame(Generation = 1:generations, Diversity = diversity)
  ggplot(diversity_data, aes(x = Generation, y = Diversity)) +
    geom_line(color = "red", size = 1) +
    labs(title = "Population Diversity Over Generations", x = "Generations", y = "Diversity") +
    theme_minimal() +
    theme(
      plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
      axis.title = element_text(size = 12)
    )
}


In [5]:
read_bin_packing_instance <- function(filename) {
  lines <- readLines(filename)
  P <- as.integer(lines[1])  # Número de instancias de problemas
  problems <- list()
  index <- 2
  for (i in 1:P) {
    problem_id <- trimws(lines[index])
    index <- index + 1
    bin_info <- strsplit(trimws(lines[index]), "\\s+")[[1]]
    bin_capacity <- as.numeric(bin_info[1])
    num_items <- as.integer(bin_info[2])
    best_known_solution <- as.integer(bin_info[3])
    index <- index + 1
    items <- numeric(num_items)
    for (j in 1:num_items) {
      items[j] <- as.numeric(trimws(lines[index]))
      index <- index + 1
    }
    problems[[i]] <- list(
      problem_id = problem_id,
      bin_capacity = bin_capacity,
      num_items = num_items,
      best_known_solution = best_known_solution,
      items = items
    )
  }
  return(problems)
}

### Definición de la clase de bin packing

In [6]:
BinPacking <- setRefClass(
  "BinPacking",
  fields = list(
    item_sizes = "numeric",
    bin_capacity = "numeric"
  ),
  methods = list(
    evaluate = function(permutation) {
      bin_capacities <- numeric() 
      current_bin_capacity <- 0  

      for (i in permutation) {
        item_size <- item_sizes[i]
        if (current_bin_capacity + item_size <= bin_capacity) {
          current_bin_capacity <- current_bin_capacity + item_size
        } else {
          bin_capacities <- c(bin_capacities, current_bin_capacity) 
          current_bin_capacity <- item_size  
        }
      }
      bin_capacities <- c(bin_capacities, current_bin_capacity)  
      return(length(bin_capacities)) 
    }
  )
)

#### Evaluación de la función

In [7]:
fitness_function <- function(permutation) {
  bp$evaluate(permutation)
}

#### Generación inicial de la población

In [8]:
generate_initial_population <- function(size, n) {
  replicate(size, sample(1:n), simplify = FALSE)
}

### Selección de individuos (roulette, rank, tournament) a través de los padres

In [9]:
select_individual <- function(fitness_vector, selection_method = "tournament", tournament_size = 3, verbose = FALSE) {
  if (!selection_method %in% c("roulette", "rank", "tournament")) {
    stop("The selection method must be 'roulette', 'rank', or 'tournament'.")
  }

  if (selection_method == "roulette") {
    selection_probability <- fitness_vector / sum(fitness_vector)
    selected_index <- sample(1:length(fitness_vector), size = 1, prob = selection_probability)
  } else if (selection_method == "rank") {
    selection_probability <- 1 / rank(-fitness_vector)
    selected_index <- sample(1:length(fitness_vector), size = 1, prob = selection_probability)
  } else if (selection_method == "tournament") {
    tournament_indices <- sample(1:length(fitness_vector), size = tournament_size)
    selected_index <- tournament_indices[which.min(fitness_vector[tournament_indices])]
  }

  if (verbose) {
    cat("----------------------", "\n")
    cat("Selected Individual", "\n")
    cat("----------------------", "\n")
    cat("Selection Method    =", selection_method, "\n")
    cat("Selected Index =", selected_index, "\n")
  }

  return(selected_index)
}


In [10]:
select_parents <- function(population, fitness, selection_method = "tournament", tournament_size = 3) {
  selected <- vector("list", length(population))
  for (i in 1:length(population)) {
    selected_index <- select_individual(fitness, selection_method, tournament_size)
    selected[[i]] <- population[[selected_index]]
  }
  return(selected)
}


### Función para introducir diversidad

In [11]:
introduce_diversity <- function(population, diversity_threshold = 0.9) {
  n <- length(population[[1]])
  unique_individuals <- length(unique(sapply(population, paste, collapse = "")))
  if (unique_individuals / length(population) < diversity_threshold) {
    num_new_individuals <- length(population) - unique_individuals
    new_individuals <- generate_initial_population(num_new_individuals, n)
    population[(unique_individuals + 1):length(population)] <- new_individuals
  }
  return(population)
}

### Métodos de crossover (uniform, one point, two point)

In [12]:

uniform_crossover <- function(parent1, parent2) {
  size <- length(parent1)
  child1 <- rep(-1, size)
  child2 <- rep(-1, size)
  used1 <- integer(0)
  used2 <- integer(0)
  
  # Selección aleatoria de genes
  for (i in seq_len(size)) {
    if (runif(1) < 0.5) {
      if (!(parent1[i] %in% used1)) {
        child1[i] <- parent1[i]
        used1 <- c(used1, parent1[i])
      }
    } else {
      if (!(parent2[i] %in% used1)) {
        child1[i] <- parent2[i]
        used1 <- c(used1, parent2[i])
      }
    }
    
    if (runif(1) < 0.5) {
      if (!(parent1[i] %in% used2)) {
        child2[i] <- parent1[i]
        used2 <- c(used2, parent1[i])
      }
    } else {
      if (!(parent2[i] %in% used2)) {
        child2[i] <- parent2[i]
        used2 <- c(used2, parent2[i])
      }
    }
  }
  
  # Rellenar los genes faltantes en child1
  missing1 <- parent1[!(parent1 %in% used1)]
  j <- 1
  for (i in seq_len(size)) {
    if (child1[i] == -1) {
      child1[i] <- missing1[j]
      j <- j + 1
    }
  }
  
  # Rellenar los genes faltantes en child2
  missing2 <- parent2[!(parent2 %in% used2)]
  j <- 1
  for (i in seq_len(size)) {
    if (child2[i] == -1) {
      child2[i] <- missing2[j]
      j <- j + 1
    }
  }
  
  return(list(child1 = child1, child2 = child2))
}

In [13]:
one_point_crossover <- function(parent1, parent2) {
  size <- length(parent1)
  cut <- sample(2:(size - 1), 1) 
  
  child1 <- c(parent1[1:cut], setdiff(parent2, parent1[1:cut]))
  child2 <- c(parent2[1:cut], setdiff(parent1, parent2[1:cut]))
  
  return(list(child1, child2))
}


In [14]:
two_point_crossover <- function(parent1, parent2) {
  size <- length(parent1)
  cuts <- sort(sample(0:(size - 1), 2))
  cut1 <- cuts[1]
  cut2 <- cuts[2]
  
  child1 <- rep(NA, size)
  child2 <- rep(NA, size)
  
  child1[(cut1 + 1):cut2] <- parent1[(cut1 + 1):cut2]
  child2[(cut1 + 1):cut2] <- parent2[(cut1 + 1):cut2]
  
  used1 <- parent1[(cut1 + 1):cut2]
  j <- (cut2 + 1) %% size
  if (j == 0) j <- size
  
  for (i in 1:size) {
    if (!(parent2[i] %in% used1)) {
      while (!is.na(child1[j])) {
        j <- (j %% size) + 1
        if (j == 0) j <- size
      }
      child1[j] <- parent2[i]
    }
  }
  
  used2 <- parent2[(cut1 + 1):cut2]
  j <- (cut2 + 1) %% size
  if (j == 0) j <- size
  
  for (i in 1:size) {
    if (!(parent1[i] %in% used2)) {
      while (!is.na(child2[j])) {
        j <- (j %% size) + 1
        if (j == 0) j <- size
      }
      child2[j] <- parent1[i]
    }
  }
  
  return(list(child1, child2))
}

### Método de mutación (adaptative, inversion, swap, insertion)

In [15]:
adaptive_mutation <- function(individual, generation, max_generations) {
  base_mutation_rate <- 0.1
  mutation_rate <- base_mutation_rate * (1 - generation / max_generations)
  if (runif(1) < mutation_rate) {
    return(swap_mutation(individual))
  } else {
    return(individual)
  }
}

In [16]:
inversion_mutation <- function(individual) {
  idx <- sort(sample(1:length(individual), 2))  
  individual[idx[1]:idx[2]] <- rev(individual[idx[1]:idx[2]]) 
  return(individual)
}

In [17]:
swap_mutation <- function(individual) {
  idx <- sample(1:length(individual), 2)
  individual[idx] <- individual[rev(idx)]
  return(individual)
}

In [18]:
insertion_mutation <- function(individual) {
  idx <- sample(1:length(individual), 2)
  item <- individual[idx[1]]
  individual <- individual[-idx[1]]
  individual <- append(individual, item, after = idx[2] - 1)
  return(individual)
}

### Algoritmo genético

In [19]:
GA <- function(bp, population_size, mutation_rate, generations, elite_ratio = 0.1, mutation_strategy = "swap", crossover_strategy = "two_point", selection_method = "tournament", tournament_size = 3, diversity_threshold = 0.9, use_diversity = TRUE) {
  n <- length(bp$item_sizes)
  population <- generate_initial_population(population_size, n)
  fitness <- sapply(population, fitness_function)
  best_fitness <- min(fitness)
  best_individual <- population[[which.min(fitness)]]
  
  best_fitness_over_time <- numeric(generations)
  population_fitness_over_time <- matrix(NA, nrow = population_size, ncol = generations)
  average_fitness_over_time <- numeric(generations)
  diversity_over_time <- numeric(generations)
  
  best_fitness_over_time[1] <- best_fitness
  population_fitness_over_time[, 1] <- fitness
  average_fitness_over_time[1] <- mean(fitness)
  diversity_over_time[1] <- length(unique(fitness))
  
  mutate <- switch(mutation_strategy,
                   "swap" = swap_mutation,
                   "insertion" = insertion_mutation,
                   "inversion" = inversion_mutation,
                   "adaptive" = function(individual) adaptive_mutation(individual, gen, generations))

  crossover <- switch(crossover_strategy,
                      "one_point" = one_point_crossover,
                      "two_point" = two_point_crossover,
                      "uniform" = uniform_crossover)
  
  for (gen in 2:generations) {
    selected_parents <- select_parents(population, fitness, selection_method, tournament_size)
    next_population <- vector("list", population_size)
    elite_size <- floor(population_size * elite_ratio)
    
    elite_indices <- order(fitness)[1:elite_size]
    next_population[1:elite_size] <- population[elite_indices]
    
    for (i in seq(elite_size + 1, population_size, by = 2)) {
      if (i + 1 > population_size) {
        next_population[[i]] <- population[[sample(1:length(population), 1)]]
        break
      }
      parents <- selected_parents[i:(i+1)]
      offspring <- crossover(parents[[1]], parents[[2]])
      if (runif(1) < mutation_rate) offspring[[1]] <- mutate(offspring[[1]])
      if (runif(1) < mutation_rate) offspring[[2]] <- mutate(offspring[[2]])
      next_population[[i]] <- offspring[[1]]
      next_population[[i+1]] <- offspring[[2]]
    }
    
    if (use_diversity) {
      population <- introduce_diversity(next_population, diversity_threshold)
    } else {
      population <- next_population
    }
    
    fitness <- sapply(population, fitness_function)
    current_best_fitness <- min(fitness)
    if (current_best_fitness < best_fitness) {
      best_fitness <- current_best_fitness
      best_individual <- population[[which.min(fitness)]]
    }
    best_fitness_over_time[gen] <- best_fitness
    population_fitness_over_time[, gen] <- fitness
    average_fitness_over_time[gen] <- mean(fitness)
    diversity_over_time[gen] <- length(unique(fitness))
  }

  return(list(best_individual = best_individual, best_fitness = best_fitness, best_fitness_over_time = best_fitness_over_time, population_fitness_over_time = population_fitness_over_time, average_fitness_over_time = average_fitness_over_time, diversity_over_time = diversity_over_time))
}


In [25]:
run_ga_multiple_times <- function(bp, num_runs = 11, population_size = 100, mutation_rate = 0.01, generations = 100) {
  results <- list()
  
  for (run in 1:num_runs) {
    execution_time <- system.time({
      result <- GA(bp, population_size, mutation_rate, generations)
    })
    
    best_individual <- result$best_individual
    best_fitness <- result$best_fitness
    best_fitness_over_time <- result$best_fitness_over_time
    population_fitness_over_time <- result$population_fitness_over_time
    average_fitness_over_time <- result$average_fitness_over_time
    diversity_over_time <- result$diversity_over_time
    
    results[[run]] <- list(
      best_individual = best_individual,
      best_fitness = best_fitness,
      best_fitness_over_time = best_fitness_over_time,
      population_fitness_over_time = population_fitness_over_time,
      average_fitness_over_time = average_fitness_over_time,
      diversity_over_time = diversity_over_time,
      execution_time = execution_time["elapsed"]
    )
    
    message("Finished run ", run, " for GA with population size ", population_size, ", mutation rate ", mutation_rate, ", and ", generations, " generations. Best fitness: ", best_fitness)
    flush.console()
  }
  
  message("Finished running GA with population size ", population_size, ", mutation rate ", mutation_rate, ", and ", generations, " generations.")
  flush.console()
  
  return(results)
}


In [27]:
library(dplyr)


Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [57]:
analyze_ga_results_df <- function(df) {
  # Extraer y verificar los datos
  best_fitnesses <- df$best_fitness
  execution_times <- df$execution_time
  
  # Calcular estadísticas
  summary_stats <- list(
    best_fitness = list(
      mean = mean(best_fitnesses),
      sd = sd(best_fitnesses),
      min = min(best_fitnesses),
      max = max(best_fitnesses)
    ),
    execution_time = list(
      mean = mean(execution_times),
      sd = sd(execution_times),
      min = min(execution_times),
      max = max(execution_times)
    )
  )
  
  return(summary_stats)
}


In [59]:
convert_to_dataframe <- function(results) {
  data_frames <- lapply(results, function(result) {
    data.frame(
      best_individual = I(list(result$best_individual)), # I() para almacenar listas dentro de data.frames
      best_fitness = result$best_fitness,
      best_fitness_over_time = I(list(result$best_fitness_over_time)),
      population_fitness_over_time = I(list(result$population_fitness_over_time)),
      average_fitness_over_time = I(list(result$average_fitness_over_time)),
      diversity_over_time = I(list(result$diversity_over_time)),
      execution_time = result$execution_time
    )
  })
  
  combined_df <- bind_rows(data_frames)
  return(combined_df)
}

## Comienzo de pruebas

In [22]:
file_path <- "./instances/binpack1.txt"
instances <- read_bin_packing_instance(file_path)


Warning message in readLines(filename):
"incomplete final line found on './instances/binpack1.txt'"


In [23]:
instance <- instances[[1]]
print(instance)

$problem_id
[1] "u120_00"

$bin_capacity
[1] 150

$num_items
[1] 120

$best_known_solution
[1] 48

$items
  [1] 42 69 67 57 93 90 38 36 45 42 33 79 27 57 44 84 86 92 46 38 85 33 82 73 49
 [26] 70 59 23 57 72 74 69 33 42 28 46 30 64 29 74 41 49 55 98 80 32 25 38 82 30
 [51] 35 39 57 84 62 50 55 27 30 36 20 78 47 26 45 41 58 98 91 96 73 84 37 93 91
 [76] 43 73 85 81 79 71 80 76 83 41 78 70 23 42 87 43 84 60 55 49 78 73 62 36 44
[101] 94 69 32 96 70 84 58 78 25 80 58 66 83 24 98 60 42 43 43 39



In [30]:
file_path <- "./instances/binpack1.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
generationsNumber <- 1000;
bp <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

instance_results = run_ga_multiple_times(bp = bp, num_runs = 11, generations = generationsNumber)

Finished run 1 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 52

Finished run 2 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 51

Finished run 3 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 52

Finished run 4 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 52

Finished run 5 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 52

Finished run 6 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 51

Finished run 7 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 51

Finished run 8 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 52

Finished run 9 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 52

Finished run 10 for GA with 

In [60]:
instance_results_df <- convert_to_dataframe(instance_results)
summary_stats <- analyze_ga_results_df(instance_results_df)
print(summary_stats)

$best_fitness
$best_fitness$mean
[1] 51.63636

$best_fitness$sd
[1] 0.504525

$best_fitness$min
[1] 51

$best_fitness$max
[1] 52


$execution_time
$execution_time$mean
[1] 67.17973

$execution_time$sd
[1] 2.896411

$execution_time$min
[1] 62.441

$execution_time$max
[1] 71.638




In [61]:
file_path <- "./instances/binpack5.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
generationsNumber <- 1000;
bp <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

instance_results = run_ga_multiple_times(bp = bp, num_runs = 11, generations = generationsNumber)
instance_results_df <- convert_to_dataframe(instance_results)
summary_stats <- analyze_ga_results_df(instance_results_df)
print(summary_stats)

Finished run 1 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 2 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 3 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 4 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 21

Finished run 5 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 21

Finished run 6 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 7 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 8 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 9 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 22

Finished run 10 for GA with 

$best_fitness
$best_fitness$mean
[1] 21.72727

$best_fitness$sd
[1] 0.4670994

$best_fitness$min
[1] 21

$best_fitness$max
[1] 22


$execution_time
$execution_time$mean
[1] 28.65782

$execution_time$sd
[1] 3.255266

$execution_time$min
[1] 24.134

$execution_time$max
[1] 32.775




In [62]:
file_path <- "./instances/instances1.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
generationsNumber <- 1000;
bp <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

instance_results = run_ga_multiple_times(bp = bp, num_runs = 11, generations = generationsNumber)
instance_results_df <- convert_to_dataframe(instance_results)
summary_stats <- analyze_ga_results_df(instance_results_df)
print(summary_stats)

Finished run 1 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 2 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 3 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 4 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 5 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 6 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 7 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 8 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 9 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 7

Finished run 10 for GA with populatio

$best_fitness
$best_fitness$mean
[1] 7

$best_fitness$sd
[1] 0

$best_fitness$min
[1] 7

$best_fitness$max
[1] 7


$execution_time
$execution_time$mean
[1] 12.05109

$execution_time$sd
[1] 1.399522

$execution_time$min
[1] 10.797

$execution_time$max
[1] 15.252




In [63]:
file_path <- "./instances/triplets.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
generationsNumber <- 1000;
bp <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

instance_results = run_ga_multiple_times(bp = bp, num_runs = 11, generations = generationsNumber)
instance_results_df <- convert_to_dataframe(instance_results)
summary_stats <- analyze_ga_results_df(instance_results_df)
print(summary_stats)

Finished run 1 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 2 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 3 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 4 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 5 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 6 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 7 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 8 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 9 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 2

Finished run 10 for GA with populatio

$best_fitness
$best_fitness$mean
[1] 2

$best_fitness$sd
[1] 0

$best_fitness$min
[1] 2

$best_fitness$max
[1] 2


$execution_time
$execution_time$mean
[1] 8.075182

$execution_time$sd
[1] 0.6002707

$execution_time$min
[1] 7.167

$execution_time$max
[1] 8.811




In [64]:
file_path <- "./instances/instances2.txt"
instances <- read_bin_packing_instance(file_path)
instance <- instances[[1]]
generationsNumber <- 1000;
bp <- BinPacking$new(item_sizes = instance$items, bin_capacity = instance$bin_capacity)

instance_results = run_ga_multiple_times(bp = bp, num_runs = 11, generations = generationsNumber)
instance_results_df <- convert_to_dataframe(instance_results)
summary_stats <- analyze_ga_results_df(instance_results_df)
print(summary_stats)

Finished run 1 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 2 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 3 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 4 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 5 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 6 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 7 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 8 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 9 for GA with population size 100, mutation rate 0.01, and 1000 generations. Best fitness: 10

Finished run 10 for GA with 

$best_fitness
$best_fitness$mean
[1] 10

$best_fitness$sd
[1] 0

$best_fitness$min
[1] 10

$best_fitness$max
[1] 10


$execution_time
$execution_time$mean
[1] 13.87555

$execution_time$sd
[1] 0.3876664

$execution_time$min
[1] 13.463

$execution_time$max
[1] 14.763


